### CHAPTER 02_머신러닝 프로젝트 처음부터 끝까지
#### HANDS-ON : https://github.com/rickiepark/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb
#### NOTION : https://juyeon-studing.notion.site/CHAPTER-02-ddcbc814b30044a3942947362243c4cc

### 2.3 데이터 가져오기

In [ ]:
import pandas as pd

head() # 처음 다섯 행 보기
tail() # 끝 다섯 행 보기
info() # 간략한 설명 (전체 행 수, 각 특성의 데이터 타입, 널값이 아닌 값의 개수 확인)
value_counts() # 범주형 변수 카테고리 확인, 카테고리마다 개수 확인
describe() # 수치형 변수 특성 요약 정보

In [ ]:
import matplotlib.pyplot as plt
hist(bins = 50, figsize = (20,15))
plt.show()

In [ ]:
#테스트 데이터 분할하기

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
#계층적 샘플링
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

### 2.4 데이터 이해를 위한 탐색과 시각화

In [ ]:
#분포(산점도) 확인
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)

#kind : 그래프 종류
#s : 원의 크기
#cmap=plt.get_cmap("jet") : 높낮이 범위를 가짐

In [ ]:
#상관관계 확인
#.corr()

#상관관계 산점도 확인
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")

### 2.5 머신러닝 알고리즘을 위한 데이터 준비

In [ ]:
# 수치형 자료 결측값 처리
dropna(subset = []) #해당 구역 삭제
drop() #전체 특성 삭제 axis = 행,열
fillna() #특정값으로 결측값 채우기

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
# 다른 방법: housing_num = housing.select_dtypes(include=[np.number])
imputer.fit(housing_num)

In [ ]:
#범주형 변수 인코딩 - OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
#범주형 변수 인코딩 - OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
#특성 스케일링 - 정규화
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()

In [ ]:
#특성 스케일링 - 표준화
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()

In [ ]:
#변환 파이프라인 - 수치형 특성을 전처리하기 위한 파이프라인

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# 하나의 변환기로 각 열마다 적절한 변환
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs), #숫자형 변환
        ("cat", OneHotEncoder(), cat_attribs), #범주형 변환
    ])

housing_prepared = full_pipeline.fit_transform(housing)

### 2.6 모델 선택과 훈련

In [ ]:
# 교차검증
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

### 2.7 모델 세부 튜닝

In [ ]:
#그리드 서치
from sklearn.model_selection import GridSearchCV

param_grid = [
    # 12(=3×4)개의 하이퍼파라미터 조합을 시도합니다.
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # bootstrap은 False로 하고 6(=2×3)개의 조합을 시도합니다.
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# 다섯 개의 폴드로 훈련하면 총 (12+6)*5=90번의 훈련이 일어납니다.
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

#최상의 파라미터 조합
grid_search.best_params_

#최적의 추정기에 직접 접근
grid_search.best_estimator_

#평가점수 확인
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
#랜덤 탐색
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    } #파라미터 개수 지정

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)


In [ ]:
#신뢰구간 계산
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))